In [1]:
import os
PREDICTIONS_FILEPATH = os.path.abspath(os.path.join("./predictions", "user_score.csv"))

In [41]:
import pandas as pd
df = pd.read_csv(PREDICTIONS_FILEPATH)
df = df.loc[df.fake_phone.drop_duplicates().index,:].reset_index(drop=True)

In [43]:
from main import db, User, Score

In [76]:
# delete all objects
_ = User.query.delete()
_ = Score.query.delete()

In [77]:
# creare users inside DB:
df['db_users'] = df.fake_phone.astype(str).apply(
    lambda x: User(phone=x)
)
# save to DB
db.session.bulk_save_objects(df.db_users.to_list())

In [78]:
# apply changes:
db.session.commit()

In [79]:
all_users = User.query.all()

In [85]:
df = df.set_index('fake_phone')

In [97]:
all_scores = list()
for u in all_users:
    s = Score(user_id = u.id, score = float(df.loc[u.phone].TARGET))
    all_scores.append(s)

In [100]:
# save Scores to DB
db.session.bulk_save_objects(all_scores)
db.session.commit()

<hr>